In [4]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run
import ecc
import helper
import tx
import script

In [2]:
from tx import Tx
from io import BytesIO
raw_tx = ('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000006b483045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed01210349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278afeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac19430600')
stream = BytesIO(bytes.fromhex(raw_tx))
transaction = Tx.parse(stream)
print(transaction.fee() >= 0)

True


In [3]:
from ecc import S256Point, Signature
sec = bytes.fromhex('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a')
der = bytes.fromhex('3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed')
z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
point = S256Point.parse(sec)
signature = Signature.parse(der)
print(point.verify(z, signature))

True


In [4]:
from helper import hash256
modified_tx = bytes.fromhex('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000001976a914a802fc56c704ce87c42d7c92eb75e7896bdc41ae88acfeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac1943060001000000')
h256 = hash256(modified_tx)
z = int.from_bytes(h256, 'big')
print(hex(z))

0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6


In [5]:
from ecc import S256Point, Signature
sec = bytes.fromhex('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a')
der = bytes.fromhex('3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed')
z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
point = S256Point.parse(sec)
signature = Signature.parse(der)
point.verify(z, signature)

True

### Exercise 1

Write the `sig_hash` method for the `Tx` class.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_sig_hash`

In [13]:
# Exercise 1

reload(tx)
run(tx.TxTest("test_sig_hash"))

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


### Exercise 2

Write the `verify_input` method for the `Tx` class. You will want to use the `TxIn.script_pubkey()`, `Tx.sig_hash()` and `Script.evaluate()` methods.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_verify_p2pkh`

In [15]:
# Exercise 2

reload(tx)
run(tx.TxTest("test_verify_p2pkh"))

.
----------------------------------------------------------------------
Ran 1 test in 0.281s

OK


In [ ]:
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx
prev_tx = bytes.fromhex('0d6fe5213c0b3291f208cba8bfb59b7476dffacc4e5cb66f6eb20a080843a299')
prev_index = 13
tx_in = TxIn(prev_tx, prev_index)
tx_outs = []
change_amount = int(0.33*100000000)
change_h160 = decode_base58('mzx5YhAH9kNHtcN481u6WkjeHjYtVeKVh2')
change_script = p2pkh_script(change_h160)
change_output = TxOut(amount=change_amount, script_pubkey=change_script)
target_amount = int(0.1*100000000)
target_h160 = decode_base58('mnrVtF8DWjMu839VW3rBfgYaAfKk8983Xf')
target_script = p2pkh_script(target_h160)
target_output = TxOut(amount=target_amount, script_pubkey=target_script)
tx_obj = Tx(1, [tx_in], [change_output, target_output], 0, True)
print(tx_obj)

In [ ]:
from ecc import PrivateKey
from helper import SIGHASH_ALL
z = transaction.sig_hash(0)
private_key = PrivateKey(secret=8675309)
der = private_key.sign(z).der()
sig = der + SIGHASH_ALL.to_bytes(1, 'big')
sec = private_key.point.sec()
script_sig = Script([sig, sec])
transaction.tx_ins[0].script_sig = script_sig 
print(transaction.serialize().hex())

In [ ]:
from ecc import PrivateKey
from helper import hash256, little_endian_to_int
secret = little_endian_to_int(hash256(b'Jimmy Song secret'))
private_key = PrivateKey(secret)
print(private_key.point.address(testnet=True))

### Exercise 3

Write the `sign_input` method for the `Tx` class.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_sign_input`

In [2]:
# Exercise 3

reload(tx)
run(tx.TxTest("test_sign_input"))

.
----------------------------------------------------------------------
Ran 1 test in 0.113s

OK


### Exercise 4

Create a testnet transaction that sends 60% of a single UTXO to `mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv`. The remaining amount minus fees should go back to your own change address. This should be a 1 input, 2 output transaction.

You can broadcast the transaction at https://testnet.blockchain.info/pushtx

In [62]:
# Exercise 4

from ecc import PrivateKey
from helper import decode_base58, SIGHASH_ALL, little_endian_to_int, hash256
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

# create 1 TxIn and 2 TxOuts
# 1 of the TxOuts should be back to your address
# the other TxOut should be to this address
target_address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'

# get the private key from the exercise in Chapter 4
# change address should be the address generated from Chapter 4
passphrase = b'negu6633@gmail.comneverevergiveup63'
secret = little_endian_to_int(hash256(passphrase))
private_key = PrivateKey(secret=secret)
address = private_key.point.address(testnet=True)

# get the prev_tx and prev_index from the transaction where you got
# some testnet coins
# create a transaction input for the previous transaction with
# the default ScriptSig and sequence
prev_tx = bytes.fromhex('4bd110db0582dc746b8afdc3d574cb65f02d37be5c2afe06f58d0d04be02a8ab')
prev_index = 1
tx_in = TxIn(prev_tx, prev_index)

# target amount should be 60% of the output amount
# set the fee to some reasonable amount
# change amount = amount from the prev tx - target amount - fee
tx_outs = []
amount = tx_in.fetch_tx(testnet=True).tx_outs[1].amount

# create a transaction output for the target amount and address
# create a transaction output for the change amount and address
# create the transaction object
change_amount = int(amount*0.39)
change_h160 = decode_base58(address)
change_script = p2pkh_script(change_h160)
change_output = TxOut(amount=change_amount, script_pubkey=change_script)
target_amount = int(amount*0.6)
target_h160 = decode_base58(target_address)
target_script = p2pkh_script(target_h160)
target_output = TxOut(amount=target_amount, script_pubkey=target_script)
tx_obj = Tx(1, [tx_in], [change_output, target_output], 0, True)

# sign the one input in the transaction object using the private key
# print the transaction's serialization in hex
# broadcast at http://testnet.blockchain.info/pushtx
z = tx_obj.sig_hash(0)
der = private_key.sign(z).der()
sig = der + SIGHASH_ALL.to_bytes(1, 'big')
sec = private_key.point.sec()
script_sig = Script([sig, sec])
tx_obj.tx_ins[0].script_sig = script_sig
print(tx_obj.serialize().hex())

0100000001aba802be040d8df506fe2a5cbe372df065cb74d5c3fd8a6b74dc8205db10d14b010000006a473044022068695d7eaa2176fd311ee04aba7ced7b9c7b95b8bb8fea1c5a1f7ef144deeee5022008affc36e811ad645fb678cbb739ebd708ec24b03fb5311967c5d89cc10c443f012103199d61763ed453c3612ac6d545f5c4381a84e64c7d10709e3068e8855e71bb45ffffffff023c0f0000000000001976a91492df3db1246b21e93c8f660eaf19df851ec8ec8b88ac70170000000000001976a914ad346f8eb57dee9a37981716e498120ae80e44f788ac00000000


### Exercise 5

Advanced: get some more testnet coins from a testnet faucet and create a 2 input, 1 output transaction. 1 input should be from the faucet, the other should be from the previous exercise, the output can be your own address.

You can broadcast the transaction at https://testnet.blockchain.info/pushtx

In [61]:
# Exercise 5

from ecc import PrivateKey
from helper import decode_base58, SIGHASH_ALL, little_endian_to_int, hash256
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

# Create 2 TxIns, 1 from the Exercise 4 and 1 from a testnet faucet
# Creat 1 TxOut to the address above
target_address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'

# get the private key from the exercise in Chapter 4
passphrase = b'negu6363'
secret = little_endian_to_int(hash256(passphrase))
private_key = PrivateKey(secret=secret)
address = private_key2.point.address(testnet=True)

# get the prev_tx and prev_index from the transaction where you got
# some testnet coins
# create the first transaction input with the default ScriptSig and
# sequence
# get the prev_tx and prev_index from the transaction in Exercise 4
# create the second transaction input with the default ScriptSig and
# sequence
prev_tx = bytes.fromhex('2c4d16eac26ed5ea39e3274bdc33750b49a1a3d62afb4260d53626814e1b1395')
prev_index = 0
tx_in = TxIn(prev_tx=prev_tx, prev_index=prev_index)
prev_tx2 = bytes.fromhex('222f7c8dab961eddde739f85a143a06a1c60fd5303bb074b933af1e2d305e71d')
prev_index2 = 0
tx_in2 = TxIn(prev_tx=prev_tx2, prev_index=prev_index2)

# set the fee to some reasonable amount
# target amount should be the sum of the inputs - fee
amount = 0

# create a transaction output for the amount and address
target_amount = int(0.00001*100000000)
target_h160 = decode_base58(target_address)
target_script = p2pkh_script(target_h160)
target_output = TxOut(amount=target_amount, script_pubkey=target_script)
tx_obj = Tx(1, [tx_in, tx_in2], [target_output], 0, True)

# sign the first input using the private key
z = tx_obj.sig_hash(0)
der = private_key.sign(z).der()
sig = der + SIGHASH_ALL.to_bytes(1, 'big')
sec = private_key.point.sec()
script_sig = Script([sig, sec])
tx_obj.tx_ins[0].script_sig = script_sig
# sign the second input using the private key
z = tx_obj.sig_hash(1)
der = private_key.sign(z).der()
sig = der + SIGHASH_ALL.to_bytes(1, 'big')
sec = private_key.point.sec()
script_sig = Script([sig, sec])
tx_obj.tx_ins[1].script_sig = script_sig

# print the transaction's serialization in hex
# broadcast at http://testnet.blockchain.info/pushtx
print(tx_obj.serialize().hex())
print(tx_obj.id())

010000000295131b4e812636d56042fb2ad6a3a1490b7533dc4b27e339ead56ec2ea164d2c000000006a47304402203e5b9c33e2a0527aba3ad1ffff601a01430909c6140d7a3decccf9c66caff7750220447ad5163ff0df206ac86656b7b49bf61b6457da1aaa46d06918a0cf0465390d012102c53788b1f1f92206e5083cf59af7321b8d39bec0d5bd41a92bd09cdda367da54ffffffff1de705d3e2f13a934b07bb0353fd601c6aa043a1859f73dedd1e96ab8d7c2f22000000006a47304402204e55ead580a75eb3e60547ae53abfbb09b363fa83e7aeec3f7b26811350bb67002200e0ad694baa45583d123f72cf819ad8f2c07035193c798a54fc4e8ae61e4d356012102c53788b1f1f92206e5083cf59af7321b8d39bec0d5bd41a92bd09cdda367da54ffffffff01e8030000000000001976a914ad346f8eb57dee9a37981716e498120ae80e44f788ac00000000
e5dc9e43f3803437b67b316be6f7c0fb31d55abcbb9cc57391fec3e4510cbad8
